# 📊 Neo4j Import and Analysis - LLM-TIKG Knowledge Graph

This notebook imports entities and relationships into Neo4j database and provides analysis tools.

## 📋 **Steps:**
1. Connect to Neo4j database
2. Load entity and relationship data from CSV files
3. Import entities with file-based labels
4. **Create all relationships from cypher scripts**
5. Verify imported data with queries


In [ ]:
# Import libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from neo4j import GraphDatabase
from pathlib import Path

print("📦 Libraries imported successfully!")


In [ ]:
# Load Neo4j connection details
load_status = load_dotenv("../Neo4j-3389ae49-Created-2025-08-31.txt")

if not load_status:
    raise RuntimeError('❌ Environment variables not loaded.')

# Get Neo4j credentials
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME") 
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE", "neo4j")

print(f"✅ Neo4j credentials loaded")
print(f"   URI: {NEO4J_URI}")
print(f"   Database: {NEO4J_DATABASE}")
print(f"   Username: {NEO4J_USERNAME}")


In [ ]:
# Neo4j Driver Class
class Neo4jDriver:
    def __init__(self, uri, username, password, database="neo4j"):
        self.uri = uri
        self.username = username
        self.password = password
        self.database = database
        self.driver = None
        
    def connect(self):
        """Connect to Neo4j database"""
        try:
            self.driver = GraphDatabase.driver(
                self.uri, 
                auth=(self.username, self.password)
            )
            # Verify connectivity
            self.driver.verify_connectivity()
            print("✅ Connected to Neo4j successfully!")
            return True
        except Exception as e:
            print(f"❌ Failed to connect to Neo4j: {e}")
            return False
    
    def run_query(self, query, parameters=None):
        """Execute Cypher query and return results"""
        if not self.driver:
            raise RuntimeError("❌ No Neo4j connection. Call connect() first.")
        
        with self.driver.session(database=self.database) as session:
            result = session.run(query, parameters or {})
            # Convert all records to list immediately to avoid ResultConsumedError
            records = []
            for record in result:
                records.append(dict(record))
            return records
    
    def run_cypher_script(self, cypher_content):
        """Execute multiple Cypher commands from script content"""
        if not self.driver:
            raise RuntimeError("❌ No Neo4j connection. Call connect() first.")
        
        # Split script into individual commands
        commands = []
        current_command = []
        
        for line in cypher_content.split('\n'):
            line = line.strip()
            # Skip comments and empty lines
            if line.startswith('//') or not line:
                continue
            
            current_command.append(line)
            
            # Check if command ends with semicolon
            if line.endswith(';'):
                command = ' '.join(current_command)
                if command.strip():
                    commands.append(command)
                current_command = []
        
        # Execute commands
        results = []
        with self.driver.session(database=self.database) as session:
            for i, command in enumerate(commands):
                try:
                    result = session.run(command)
                    # Convert to list to consume result
                    records = [dict(record) for record in result]
                    results.append({
                        'command_index': i,
                        'command': command[:100] + '...' if len(command) > 100 else command,
                        'success': True,
                        'records': records,
                        'count': len(records)
                    })
                except Exception as e:
                    results.append({
                        'command_index': i,
                        'command': command[:100] + '...' if len(command) > 100 else command,
                        'success': False,
                        'error': str(e),
                        'count': 0
                    })
        
        return results
    
    def close(self):
        """Close Neo4j connection"""
        if self.driver:
            self.driver.close()
            print("✅ Neo4j connection closed")

# Initialize driver
neo4j_driver = Neo4jDriver(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD, NEO4J_DATABASE)

# Connect to database
if neo4j_driver.connect():
    print("🚀 Ready to import data!")
else:
    raise RuntimeError("❌ Cannot proceed without database connection")


## 📁 **Load Data from CSV Files**


In [ ]:
# Data Loader Class
class DataLoader:
    def __init__(self, entities_dir="../data/entities"):
        self.entities_dir = entities_dir
        self.entity_data = {}
        self.relationships_data = None
        
    def load_all_entities(self):
        """Load all entity CSV files"""
        entity_types = ['attackers', 'malware', 'tools', 'vulnerabilities', 'files', 'hashes', 'ips', 'urls', 'devices', 'techniques']
        
        print("📥 Loading entity data...")
        total_entities = 0
        
        for entity_type in entity_types:
            file_path = f"{self.entities_dir}/{entity_type}.csv"
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                self.entity_data[entity_type] = df
                count = len(df)
                total_entities += count
                print(f"   ✅ {entity_type}: {count} entities")
            else:
                print(f"   ⚠️ {entity_type}: file not found")
                self.entity_data[entity_type] = pd.DataFrame()
        
        print(f"📊 Total entities loaded: {total_entities}")
        return self.entity_data
    
    def load_relationships(self, file_path="../data/relationships_norm.csv"):
        """Load relationships data"""
        if os.path.exists(file_path):
            self.relationships_data = pd.read_csv(file_path)
            count = len(self.relationships_data)
            print(f"🔗 Loaded {count} relationships")
            return self.relationships_data
        else:
            print(f"❌ Relationships file not found: {file_path}")
            return None

# Load data
data_loader = DataLoader()
entities = data_loader.load_all_entities()
relationships = data_loader.load_relationships()

print("\\n📋 Data loading complete!")


## 🏷️ **Import Entities with File-based Labels**


In [ ]:
# Import entities with labels based on CSV filenames
def import_entities_with_labels():
    """Import all entities with labels derived from CSV filenames"""
    print("🚀 Starting entity import with file-based labels...")
    print("📥 Importing entities with file-based labels...")
    
    total_imported = 0
    
    for entity_type, df in entities.items():
        if df.empty:
            continue
            
        # Convert entity_type to proper label (attackers -> Attackers)
        label = entity_type.capitalize()
        
        print(f"   📂 Importing {entity_type} entities...")
        
        try:
            # Create constraint for unique IDs
            constraint_query = f"CREATE CONSTRAINT IF NOT EXISTS FOR (n:{label}) REQUIRE n.id IS UNIQUE"
            neo4j_driver.run_query(constraint_query)
            
            # Import entities in batches
            batch_size = 100
            imported_count = 0
            
            for i in range(0, len(df), batch_size):
                batch = df.iloc[i:i+batch_size]
                
                # Prepare batch data
                entities_list = []
                for _, row in batch.iterrows():
                    entities_list.append({
                        'id': row['id'],
                        'name': row['name'],
                        'type': row['type']
                    })
                
                # Batch import query
                import_query = f"""
                UNWIND $entities AS entity
                MERGE (n:{label} {{id: entity.id}})
                SET n.name = entity.name, n.type = entity.type
                RETURN count(n) as imported_count
                """
                
                result = neo4j_driver.run_query(import_query, {'entities': entities_list})
                batch_imported = result[0]['imported_count'] if result else 0
                imported_count += batch_imported
            
            # Verify import
            count_query = f"MATCH (n:{label}) RETURN count(n) as count"
            count_result = neo4j_driver.run_query(count_query)
            final_count = count_result[0]['count'] if count_result else 0
            
            print(f"      ✅ Imported {imported_count} {entity_type} (Total in DB: {final_count})")
            total_imported += imported_count
            
        except Exception as e:
            print(f"      ❌ Error importing {entity_type}: {e}")
    
    print(f"\\n✅ Total entities imported: {total_imported}")
    return total_imported

# Import entities
import_entities_with_labels()


## 🔗 **Create All Relationships from Cypher Scripts**

**⚠️ Note:** This step requires entities to be imported first (previous cell).

This section runs all relationship creation scripts from the `create_relationships_cypher/` directory.


In [ ]:
# Create all relationships from cypher scripts
def create_all_relationships():
    """Execute all relationship creation scripts"""
    
    script_dir = Path("../cypher_queries/create_relationships_cypher")
    
    # Define script execution order (from smallest to largest for efficiency)
    script_order = [
        "create_vulnerabilities_relationships.cypher",
        "create_attackers_relationships.cypher", 
        "create_files_relationships.cypher",
        "create_tools_relationships.cypher",
        "create_malware_relationships.cypher"
    ]
    
    print("🔗 Starting relationship creation from cypher scripts...")
    print("=" * 60)
    
    total_relationships_created = 0
    execution_summary = []
    
    for script_name in script_order:
        script_path = script_dir / script_name
        
        if not script_path.exists():
            print(f"⚠️ Script not found: {script_name}")
            continue
            
        print(f"\\n📄 Executing: {script_name}")
        print("-" * 40)
        
        try:
            # Read script content
            with open(script_path, 'r', encoding='utf-8') as f:
                script_content = f.read()
            
            # Execute script
            results = neo4j_driver.run_cypher_script(script_content)
            
            # Process results
            successful_commands = 0
            failed_commands = 0
            script_relationships = 0
            
            for result in results:
                if result['success']:
                    successful_commands += 1
                    # Count relationships created (MERGE commands)
                    if 'MERGE' in result['command'].upper():
                        script_relationships += 1
                else:
                    failed_commands += 1
                    print(f"   ❌ Failed command: {result['command']}")
                    print(f"      Error: {result['error']}")
            
            total_relationships_created += script_relationships
            
            execution_summary.append({
                'script': script_name,
                'successful_commands': successful_commands,
                'failed_commands': failed_commands,
                'relationships_created': script_relationships
            })
            
            print(f"   ✅ Commands executed: {successful_commands}")
            print(f"   ❌ Commands failed: {failed_commands}")
            print(f"   🔗 Relationships created: {script_relationships}")
            
        except Exception as e:
            print(f"   ❌ Error executing {script_name}: {e}")
            execution_summary.append({
                'script': script_name,
                'successful_commands': 0,
                'failed_commands': 1,
                'relationships_created': 0,
                'error': str(e)
            })
    
    print(f"\\n" + "=" * 60)
    print("📊 RELATIONSHIP CREATION SUMMARY:")
    print("=" * 60)
    
    for summary in execution_summary:
        status = "✅" if summary['failed_commands'] == 0 else "⚠️"
        print(f"{status} {summary['script']}")
        print(f"   Commands: {summary['successful_commands']} success, {summary['failed_commands']} failed")
        print(f"   Relationships: {summary['relationships_created']}")
        if 'error' in summary:
            print(f"   Error: {summary['error']}")
        print()
    
    print(f"🎯 TOTAL RELATIONSHIPS CREATED: {total_relationships_created}")
    
    # Verify final count
    verify_query = "MATCH ()-[r]->() RETURN count(r) as total_relationships"
    verify_result = neo4j_driver.run_query(verify_query)
    actual_count = verify_result[0]['total_relationships'] if verify_result else 0
    
    print(f"📊 TOTAL RELATIONSHIPS IN DATABASE: {actual_count}")
    
    return execution_summary, total_relationships_created

# Execute relationship creation
summary, created_count = create_all_relationships()


## ✅ **Verify Relationship Creation Results**


In [ ]:
# Verify relationship creation results
def verify_relationships():
    """Verify that relationships were created successfully"""
    print("🔍 VERIFYING RELATIONSHIP CREATION:")
    print("=" * 50)
    
    # 1. Count all relationships by type
    print("\\n1️⃣ Relationships by Type:")
    type_query = """
    MATCH ()-[r]->()
    RETURN type(r) AS relationship_type, count(r) AS count
    ORDER BY count DESC
    LIMIT 15
    """
    
    type_results = neo4j_driver.run_query(type_query)
    for result in type_results:
        print(f"   {result['relationship_type']}: {result['count']}")
    
    # 2. Count relationships by source entity type
    print("\\n2️⃣ Relationships by Source Entity Type:")
    source_query = """
    MATCH (source)-[r]->()
    RETURN labels(source)[0] AS source_type, count(r) AS count
    ORDER BY count DESC
    """
    
    source_results = neo4j_driver.run_query(source_query)
    for result in source_results:
        print(f"   {result['source_type']}: {result['count']}")
    
    # 3. Sample relationships
    print("\\n3️⃣ Sample Relationships:")
    sample_query = """
    MATCH (source)-[r]->(target)
    RETURN labels(source)[0] AS source_type, source.name AS source,
           type(r) AS relationship, labels(target)[0] AS target_type, target.name AS target
    ORDER BY source_type, relationship
    LIMIT 10
    """
    
    sample_results = neo4j_driver.run_query(sample_query)
    for result in sample_results:
        print(f"   {result['source']} ({result['source_type']}) → {result['relationship']} → {result['target']} ({result['target_type']})")
    
    # 4. Top connected entities
    print("\\n4️⃣ Most Connected Entities:")
    connected_query = """
    MATCH (n)
    OPTIONAL MATCH (n)-[r_out]->()
    OPTIONAL MATCH ()-[r_in]->(n)
    WITH n, count(r_out) + count(r_in) AS total_connections
    WHERE total_connections > 0
    RETURN labels(n)[0] AS entity_type, n.name AS entity, total_connections
    ORDER BY total_connections DESC
    LIMIT 10
    """
    
    connected_results = neo4j_driver.run_query(connected_query)
    for result in connected_results:
        print(f"   {result['entity']} ({result['entity_type']}): {result['total_connections']} connections")
    
    # 5. Final statistics
    print("\\n5️⃣ Final Statistics:")
    stats_queries = [
        ("Total Nodes", "MATCH (n) RETURN count(n) as count"),
        ("Total Relationships", "MATCH ()-[r]->() RETURN count(r) as count"),
        ("Unique Relationship Types", "MATCH ()-[r]->() RETURN count(DISTINCT type(r)) as count"),
        ("Unique Node Labels", "CALL db.labels() YIELD label RETURN count(label) as count")
    ]
    
    for stat_name, query in stats_queries:
        result = neo4j_driver.run_query(query)
        count = result[0]['count'] if result else 0
        print(f"   {stat_name}: {count}")
    
    print("\\n✅ Relationship verification completed!")

# Run verification
verify_relationships()


In [ ]:
# Close Neo4j connection
neo4j_driver.close()
print("🔒 Neo4j connection closed.")
print("✅ Relationship creation and verification completed successfully!")


# Neo4j Database Connection and Data Import

This notebook connects to Neo4j database and imports extracted entities and relationships for threat intelligence knowledge graph construction.

## Overview
- **Database Connection**: Connect to Neo4j Aura instance
- **Data Import**: Import entities and relationships from CSV files
- **Graph Analysis**: Analyze the constructed knowledge graph
- **Query Examples**: Sample Cypher queries for threat intelligence analysis

## Prerequisites
- Neo4j Aura instance credentials
- Extracted entities and relationships CSV files
- Python packages: `neo4j`, `python-dotenv`, `pandas`


In [1]:
# # Import required libraries
# import os
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from neo4j import GraphDatabase
# from dotenv import load_dotenv
# import warnings
# warnings.filterwarnings('ignore')
#
# # Set plotting style
# plt.style.use('default')
# sns.set_palette("husl")
# plt.rcParams['figure.figsize'] = (12, 8)
# plt.rcParams['font.size'] = 12
#
# print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


## 1. Database Connection Setup


In [2]:
# # Load environment variables from Neo4j credentials file
# load_status = load_dotenv("../Neo4j-3389ae49-Created-2025-08-31.txt")
# if load_status is False:
#     raise RuntimeError('Environment variables not loaded.')
#
# # Get Neo4j connection details
# URI = os.getenv("NEO4J_URI")
# USERNAME = os.getenv("NEO4J_USERNAME")
# PASSWORD = os.getenv("NEO4J_PASSWORD")
# DATABASE = os.getenv("NEO4J_DATABASE", "neo4j")
#
# print(f"🔗 Connecting to Neo4j database...")
# print(f"   URI: {URI}")
# print(f"   Database: {DATABASE}")
# print(f"   Username: {USERNAME}")


🔗 Connecting to Neo4j database...
   URI: neo4j+s://3389ae49.databases.neo4j.io
   Database: neo4j
   Username: neo4j


In [3]:
# # Create Neo4j driver class
# class Neo4jDriver:
#     def __init__(self, uri, username, password, database):
#         self.uri = uri
#         self.username = username
#         self.password = password
#         self.database = database
#         self.driver = None
#
#     def connect(self):
#         """Connect to Neo4j database"""
#         try:
#             self.driver = GraphDatabase.driver(self.uri, auth=(self.username, self.password))
#             self.driver.verify_connectivity()
#             print("✅ Connection established successfully!")
#
#             # Get database info
#             with self.driver.session(database=self.database) as session:
#                 result = session.run("CALL dbms.components() YIELD name, versions, edition")
#                 for record in result:
#                     print(f"   Neo4j {record['name']} {record['versions'][0]} ({record['edition']})")
#
#             return True
#
#         except Exception as e:
#             print(f"❌ Connection failed: {e}")
#             return False
#
#     def close(self):
#         """Close Neo4j connection"""
#         if self.driver:
#             self.driver.close()
#             print("✅ Connection closed.")
#
#     def run_query(self, query, parameters=None):
#         """Run a Cypher query and return all records as list of dicts"""
#         try:
#             with self.driver.session(database=self.database) as session:
#                 result = session.run(query, parameters or {})
#                 # Convert to list of dicts immediately to avoid Result object issues
#                 records = []
#                 for record in result:
#                     records.append(dict(record))
#                 return records
#         except Exception as e:
#             print(f"❌ Query failed: {e}")
#             return []
#
# # Create Neo4j driver instance
# neo4j_driver = Neo4jDriver(URI, USERNAME, PASSWORD, DATABASE)
#
# # Test connection
# if neo4j_driver.connect():
#     print("\n✅ Neo4j driver initialized and ready to use!")


✅ Connection established successfully!
   Neo4j Neo4j Kernel 5.27-aura (enterprise)
   Neo4j Cypher 5 ()

✅ Neo4j driver initialized and ready to use!


## 2. Load Data Files


In [4]:
# # Define data loading class
# class DataLoader:
#     def __init__(self, data_dir="../data"):
#         self.data_dir = data_dir
#         self.entities_dir = f"{data_dir}/entities"
#         self.relationships_file = f"{data_dir}/relationships_norm.csv"
#
#         self.entity_files = {
#             'attackers': 'attackers.csv',
#             'malware': 'malware.csv',
#             'tools': 'tools.csv',
#             'vulnerabilities': 'vulnerabilities.csv',
#             'files': 'files.csv',
#             'urls': 'urls.csv',
#             'techniques': 'techniques.csv',
#             'hashes': 'hashes.csv',
#             'ips': 'ips.csv',
#             'devices': 'devices.csv'
#         }
#
#         self.entity_dataframes = {}
#         self.relationships_df = None
#         self.total_entities = 0
#
#     def load_entities(self):
#         """Load entity CSV files"""
#         print("📂 Loading entity files...")
#
#         for entity_type, filename in self.entity_files.items():
#             file_path = f"{self.entities_dir}/{filename}"
#             try:
#                 df = pd.read_csv(file_path)
#                 self.entity_dataframes[entity_type] = df
#                 count = len(df)
#                 self.total_entities += count
#                 print(f"   ✅ {entity_type}: {count:,} entities")
#             except FileNotFoundError:
#                 print(f"   ⚠️  {entity_type}: File not found")
#             except Exception as e:
#                 print(f"   ❌ {entity_type}: Error loading - {e}")
#
#         print(f"\n📊 Total entities loaded: {self.total_entities:,}")
#         return self.entity_dataframes
#
#     def load_relationships(self):
#         """Load relationships CSV file"""
#         print(f"\n🔗 Loading relationships...")
#         try:
#             self.relationships_df = pd.read_csv(self.relationships_file)
#             print(f"   ✅ Relationships: {len(self.relationships_df):,} relationships")
#             print(f"   ✅ Unique relationship types: {self.relationships_df['relation'].nunique()}")
#         except FileNotFoundError:
#             print(f"   ❌ Relationships file not found: {self.relationships_file}")
#         except Exception as e:
#             print(f"   ❌ Error loading relationships: {e}")
#
#         return self.relationships_df
#
# # Create data loader instance and load data
# data_loader = DataLoader()
# entity_dataframes = data_loader.load_entities()
# relationships_df = data_loader.load_relationships()


📂 Loading entity files...
   ✅ attackers: 26 entities
   ✅ malware: 129 entities
   ✅ tools: 288 entities
   ✅ vulnerabilities: 27 entities
   ✅ files: 155 entities
   ✅ urls: 23 entities
   ✅ techniques: 40 entities
   ✅ hashes: 4 entities
   ✅ ips: 7 entities
   ✅ devices: 27 entities

📊 Total entities loaded: 726

🔗 Loading relationships...
   ✅ Relationships: 554 relationships
   ✅ Unique relationship types: 87


## 3. Import Entities with File-Based Labels


In [5]:
# # Import entities with labels matching CSV file names
# def import_entities_with_file_labels(neo4j_driver, entity_dataframes):
#     """Import entities using CSV file names as Neo4j labels"""
#     print("📥 Importing entities with file-based labels...")
#
#     total_imported = 0
#
#     for entity_type, df in entity_dataframes.items():
#         if len(df) == 0:
#             continue
#
#         print(f"\n   📂 Importing {entity_type} entities...")
#
#         # Use entity_type (file name) as the label
#         label = entity_type.title()  # Capitalize first letter
#
#         # Create nodes for this entity type
#         create_query = f"""
#         UNWIND $entities AS entity
#         MERGE (n:{label} {{id: entity.id}})
#         SET n.name = entity.name,
#             n.type = entity.type,
#             n.label = '{label}',
#             n.imported_at = datetime()
#         """
#
#         # Prepare data for import
#         entities_data = df.to_dict('records')
#
#         try:
#             # Execute create query
#             neo4j_driver.run_query(create_query, {"entities": entities_data})
#
#             # Count created nodes
#             count_query = f"MATCH (n:{label}) RETURN count(n) as count"
#             count_result = neo4j_driver.run_query(count_query)
#             if count_result and len(count_result) > 0:
#                 imported_count = count_result[0]['count']
#                 print(f"      ✅ Imported {imported_count:,} {entity_type} entities (Label: {label})")
#                 total_imported += imported_count
#             else:
#                 print(f"      ⚠️  Could not count {entity_type} entities")
#
#         except Exception as e:
#             print(f"      ❌ Error importing {entity_type}: {e}")
#
#     print(f"\n✅ Total entities imported: {total_imported:,}")
#     return total_imported
#
# # Run entity import with file-based labels
# print("🚀 Starting entity import with file-based labels...")
# imported_entities = import_entities_with_file_labels(neo4j_driver, entity_dataframes)


🚀 Starting entity import with file-based labels...
📥 Importing entities with file-based labels...

   📂 Importing attackers entities...
      ✅ Imported 26 attackers entities (Label: Attackers)

   📂 Importing malware entities...
      ✅ Imported 129 malware entities (Label: Malware)

   📂 Importing tools entities...
      ✅ Imported 288 tools entities (Label: Tools)

   📂 Importing vulnerabilities entities...
      ✅ Imported 27 vulnerabilities entities (Label: Vulnerabilities)

   📂 Importing files entities...
      ✅ Imported 155 files entities (Label: Files)

   📂 Importing urls entities...
      ✅ Imported 23 urls entities (Label: Urls)

   📂 Importing techniques entities...
      ✅ Imported 40 techniques entities (Label: Techniques)

   📂 Importing hashes entities...
      ✅ Imported 4 hashes entities (Label: Hashes)

   📂 Importing ips entities...
      ✅ Imported 7 ips entities (Label: Ips)

   📂 Importing devices entities...
      ✅ Imported 27 devices entities (Label: Devices

## 4. Verify Entity Import


In [6]:
# # Verify import by checking all labels
# print("🔍 Verifying imported entities by label...")
# verification_query = """
# MATCH (n)
# WITH labels(n)[0] as Label, count(n) as Count
# RETURN Label, Count
# ORDER BY Count DESC
# """
#
# result = neo4j_driver.run_query(verification_query)
# if result and len(result) > 0:
#     print("   📊 Entities by label:")
#     for record in result:
#         print(f"      {record['Label']}: {record['Count']:,} entities")
# else:
#     print("   ❌ Could not verify entities")


🔍 Verifying imported entities by label...
   📊 Entities by label:
      Tools: 288 entities
      Files: 155 entities
      Malware: 129 entities
      Techniques: 40 entities
      Vulnerabilities: 27 entities
      Devices: 27 entities
      Attackers: 26 entities
      Urls: 23 entities
      Ips: 7 entities
      Hashes: 4 entities
